In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime, timedelta
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, ICRS
from ctapipe.coordinates import CameraFrame
from lstchain.reco.utils import clip_alt
import sys, os, glob, re, tables
import pandas as pd
pd.set_option("display.max_columns", None) #



import utils

In [2]:
run_number = 3599

source_name = "Crab"

input_path_dl1 = None # Set to None if you want to use the standard one in IT cluster
input_path_dl1_scaled = f"/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl1_merged_scaled/dl1_LST-1.Run{run_number:05}.h5"

input_path_dl2 = f"/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl2/dl2_LST-1.Run{run_number:05}.h5"
input_path_dl2_scaled = f"/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl2_scaled/dl2_LST-1.Run{run_number:05}.h5"

input_path_dl3 = f"/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl3_total/dl3_LST-1.Run{run_number:05}.fits"
input_path_dl3_scaled = f"/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl3_scaled_total/dl3_LST-1.Run{run_number:05}.fits"

input_path_dl2_radec = f"/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl2_radec/dl2_LST-1.Run{run_number:05}.h5"
input_path_dl2_radec_scaled = f"/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/dl2_scaled_radec/dl2_LST-1.Run{run_number:05}.h5"


In [3]:
# # Hardcoded information
# # Location of LST-1
# location_lst = EarthLocation.from_geodetic(-17.89139 * u.deg, 28.76139 * u.deg, 2184 * u.m)
# # Focal length of LST-1
# focal_length_lst = 28 * u.m

In [10]:
tab = table_dl2[:10]
tab

,obs_id,event_id,intensity,log_intensity,x,y,r,phi,length,length_uncertainty,width,width_uncertainty,psi,skewness,kurtosis,time_gradient,intercept,leakage_intensity_width_1,leakage_intensity_width_2,leakage_pixels_width_1,leakage_pixels_width_2,n_pixels,concentration_cog,concentration_core,concentration_pixel,n_islands,alt_tel,az_tel,calibration_id,dragon_time,ucts_time,tib_time,ucts_jump,mc_type,wl,tel_id,tel_pos_x,tel_pos_y,tel_pos_z,trigger_type,ucts_trigger_type,trigger_time,event_type,delta_t,sin_az_tel,log_reco_energy,reco_energy,reco_disp_norm,reco_disp_sign,reco_disp_sign_proba,reco_disp_dx,reco_disp_dy,reco_src_x,reco_src_y,signed_time_gradient,signed_skewness,reco_alt,reco_az,gammaness,reco_type
0,3599,1,10653.177474,4.027479,-0.223599,-0.351230,0.416364,-2.137706,1.200157,0.007111,0.938955,0.005709,1.037803,0.803724,2.495632,-11.100325,23.641275,0.085077,0.197396,0.027493,0.071159,887,0.001876,0.327765,0.008568,25,1.271877,4.362672,0,1.613082e+09,0,0,False,-9999,0.782360,1,50.0,50.0,16.0,1,1,1.613082e+09,32,0.000000,-0.93947,0.915436,8.230680,0.945150,1,0.390407,0.480243,0.814049,0.256644,0.462818,11.100325,-0.803724,1.280218,4.417818,0.007868,101
1,3599,2,677.069354,2.830633,-0.240818,0.499696,0.554698,2.019884,0.452871,0.013837,0.261210,0.006491,0.891308,1.343766,3.528272,10.675971,25.023466,0.033719,0.046693,0.001617,0.002156,40,0.000000,0.492077,0.079931,5,1.271877,4.362672,0,1.613082e+09,0,0,False,-9999,0.576788,1,50.0,50.0,16.0,1,1,1.613082e+09,32,0.000066,-0.93947,-0.325695,0.472394,0.697778,-1,0.852473,-0.438480,-0.542798,-0.679298,-0.043102,10.675971,1.343766,1.248694,4.358028,0.002402,101
2,3599,3,54.394238,1.735553,0.786361,0.498402,0.931004,0.564908,0.066771,0.004285,0.029877,0.002323,0.416841,-0.142476,1.896124,-19.131845,18.103896,0.000000,0.000000,0.000000,0.000000,4,0.911258,0.455021,0.455021,1,1.271877,4.362672,0,1.613082e+09,0,0,False,-9999,0.447458,1,50.0,50.0,16.0,1,1,1.613082e+09,32,0.000426,-0.93947,-1.507567,0.031077,0.398808,1,0.348026,0.364659,0.161467,1.151020,0.659869,19.131845,0.142476,1.310212,4.450104,0.404411,101
3,3599,7,199.547716,2.300047,0.159369,-0.038283,0.163902,-0.235750,0.257533,0.012433,0.061280,0.002725,1.288595,-1.277250,2.860304,0.283736,18.893337,0.000000,0.000000,0.000000,0.000000,11,0.156552,0.363102,0.264680,2,1.271877,4.362673,0,1.613082e+09,0,0,False,-9999,0.237951,1,50.0,50.0,16.0,1,1,1.613082e+09,32,0.000144,-0.93947,-1.303657,0.049699,0.330081,1,0.110703,0.091918,0.317024,0.251286,0.278741,-0.283736,1.277250,1.280301,4.395884,0.316970,101
4,3599,8,523.983572,2.719318,-0.839148,0.079431,0.842899,3.047218,0.247758,0.005863,0.074852,0.002421,0.809345,-0.240985,2.173712,-0.757970,24.281516,0.000000,0.012564,0.000000,0.000539,30,0.167488,0.249300,0.077829,1,1.271877,4.362673,0,1.613082e+09,0,0,False,-9999,0.302117,1,50.0,50.0,16.0,1,1,1.613082e+09,32,0.000175,-0.93947,-1.010316,0.097653,0.390791,1,0.227377,0.269635,0.282868,-0.569513,0.362299,0.757970,0.240985,1.252210,4.402144,0.239810,101
5,3599,9,117.609154,2.070441,0.266011,-0.043719,0.269579,-0.162895,0.064818,0.003111,0.045902,0.003285,-0.476861,0.251078,2.083836,-10.986920,21.154742,0.000000,0.000000,0.000000,0.000000,7,0.782281,0.360114,0.360114,1,1.271877,4.362673,0,1.613082e+09,0,0,False,-9999,0.708172,1,50.0,50.0,16.0,1,1,1.613082e+09,32,0.000196,-0.93947,-1.474158,0.033562,0.230101,-1,0.658625,-0.204431,0.105615,0.061579,0.061896,-10.986920,0.251078,1.273971,4.369894,0.206358,101
6,3599,10,1733.904834,3.239025,-0.587683,-0.666005,0.888220,-2.293802,0.432045,0.007254,0.216879,0.003226,0.203388,0.879857,2.955231,3.711894,25.114071,0.130681,0.289732,0.005930,0.014016,119,0.054651,0.382822,0.027159,1,1.271877,4.362673,0,1.613082e+09,0,0,False,-9999,0.501982,1,50.0,50.0,16.0,1,1,1.613082e+09,32,0.000029,-0.93947,-0.397252,0.400634,0.400172,-1,0.714327,-0.391924,-0.080830,-0.979607,-0.746835,3.711894,0.879857,1.237500,4.284750,0.008016,101
7,3599,11,624.014009,2.795194,-0.647941,0.323457,0.724190,2.678579,0.30894

In [6]:
# Getting the DL1 filename for the file and the datacheck as well
path_dl1        = utils.find_dl1_fname(run_number) if input_path_dl1 == None else input_path_dl1
path_dl1_dcheck = utils.find_dl1_fname(run_number, dchecking=True, print_details=False)

path_dl2, path_dl2_radec, path_dl3 = input_path_dl2, input_path_dl2_radec, input_path_dl3
path_dl1_scaled, path_dl2_scaled = input_path_dl1_scaled, input_path_dl2_scaled #
path_dl2_radec_scaled, path_dl3_scaled = input_path_dl2_radec_scaled, input_path_dl3_scaled #

# utils.check_files_exist(global_variables=globals()) # Checking if all files exist

# Opening the tables for DL1 and DL2 as well as for the datacheck
table_dl1_datacheck = tables.open_file(path_dl1_dcheck).root.dl1datacheck
table_dl1 = pd.read_hdf(path_dl1, "/dl1/event/telescope/parameters/LST_LSTCam")
table_dl1_scaled = pd.read_hdf(path_dl1_scaled, "/dl1/event/telescope/parameters/LST_LSTCam") #
table_dl2 = pd.read_hdf(path_dl2, "/dl2/event/telescope/parameters/LST_LSTCam")
table_dl2_scaled = pd.read_hdf(path_dl2_scaled, "/dl2/event/telescope/parameters/LST_LSTCam") #

DL1 file version: v0.9


In [ ]:
%%time
print(f"Adding RA and DEC data to DL2 tables... (takes ~35min)")
add_ra_dec_to_dl2(table_dl2, location_lst, focal_length_lst)
# add_ra_dec_to_dl2(table_dl2_scaled, location_lst, focal_length_lst)

In [8]:
table_dl2["obs_id"][0]

3599

In [ ]:
def add_ra_dec_to_dl2(table_dl2, location_lst, focal_length_lst, overwrite=False):

    # Extracting the run number
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    # Temporal folder for storing tables
    tmp_h5_folder = "./objects/tmp_h5/"
    tmp_fname = f"dl2_LST-1.Run{run_number:05}.h5"
    
    # We create a temporal folder for only computing the files one time
    os.makedirs(tmp_h5_folder, exist_ok=True)
    
    # Checking inside te temporal folder in case one of the tables have been already computed
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # tmp_h5_folder + tmp_fname
    
    file_exists = False
    
    if (not file_exists) or overwrite:
    
        # Observation time
        obstime_dl2 = pd.to_datetime(table_dl2["dragon_time"], unit="s")
        
        # Coordinate frame
        horizon_frame = AltAz(location=location_lst, obstime=obstime_dl2)
        
        # Telescope pointing
        pointing_alt = u.Quantity(table_dl2["alt_tel"], u.rad, copy=False)
        pointing_az  = u.Quantity(table_dl2["az_tel"], u.rad, copy=False)
        
        pointing_direction = SkyCoord(alt=clip_alt(pointing_alt), az=pointing_az, frame=horizon_frame)
        pointing_direction_icrs = pointing_direction.transform_to("icrs")
        
        # Defining the camera coordinates frame
        camera_frame = CameraFrame(
            focal_length=focal_length_lst,
            telescope_pointing=pointing_direction,
            obstime=obstime_dl2,
            location=location_lst,
        )
        
        camera_coords = SkyCoord(x=table_dl2["reco_src_x"], y=table_dl2["reco_src_y"], frame=camera_frame, unit=(u.m, u.m))
        radec_coords = camera_coords.transform_to(frame=ICRS)
        
        # adding the ra-dec coordinates of pointing and events for lst data
        table_dl2.loc[:,'reco_ra']      = radec_coords.ra.deg
        table_dl2.loc[:,'reco_dec']     = radec_coords.dec.deg 
        table_dl2.loc[:,'pointing_ra']  = pointing_direction_icrs.ra.deg
        table_dl2.loc[:,'pointing_dec'] = pointing_direction_icrs.dec.deg    
        
        # setting run-event as indexes
        table_dl2.set_index(['obs_id', 'event_id'], inplace=True)
        table_dl2.sort_index(inplace=True)

        # Then we temporally store it
        
        # table_dl2.to_hdf(tmp_h5_folder + f"dl2_LST-1.Run{run_number:05}.h5")

    # else:
        # Otherwise we only read the table
        
    return table_dl2

In [5]:
table_dl2

NameError: name 'table_dl2' is not defined

In [ ]:
%%time

# reading the dataframes
tmp_mean_dfs    = [pd.read_hdf(file, key='/events/parameters') for file in mean_files]
tmp_melibea_dfs = [pd.read_hdf(file, key='/events/parameters') for file in melibea_files]
tmp_lst_dfs     = [pd.read_hdf(file, key='/events/parameters') for file in lst_files]

# iterating over lst ones
for tmp_df in tmp_lst_dfs:

   

# appending all dataframes in a unique array
all_dfs = [*tmp_mean_dfs, *tmp_melibea_dfs, *tmp_lst_dfs] 

# deleting temporal dataframes
del tmp_lst_dfs, tmp_mean_dfs, tmp_melibea_dfs


In [ ]:
# Computing ra and dec and store it in the DL2 table



In [ ]:
# Reading DL1 and DL2 data

In [ ]:
# Reading datacheck data
# Direct quantities
srun_number = table_dl1_datacheck.cosmics.col("subrun_index")
srun_elapsed_time = table_dl1_datacheck.cosmics.col("elapsed_time")
srun_cumulative_time = np.cumsum(srun_elapsed_time)
srun_num_events = table_dl1_datacheck.cosmics.col("num_events")
srun_num_cleaned_events = table_dl1_datacheck.cosmics.col("num_cleaned_events")
srun_mean_az = np.rad2deg(table_dl1_datacheck.cosmics.col("mean_az_tel"))
srun_tel_ra  = table_dl1_datacheck.cosmics.col("tel_ra")
srun_tel_dec = table_dl1_datacheck.cosmics.col("tel_dec")

# Some manipulations
srun_dragon_time = np.array([f[0] for f in table_dl1_datacheck.cosmics.col("dragon_time")])
srun_charge_mean = np.array([np.mean(f) for f in table_dl1_datacheck.cosmics.col("charge_mean")])
srun_charge_stdv = np.array([np.mean(f) for f in table_dl1_datacheck.cosmics.col("charge_stddev")])
srun_mean_zd = 90 - np.rad2deg(table_dl1_datacheck.cosmics.col("mean_alt_tel"))
srun_num_non_cleaned_events = srun_num_events - srun_num_cleaned_events

# Calculations
srun_rate_events = srun_num_events / srun_elapsed_time
srun_rate_cleaned_events = srun_num_cleaned_events / srun_elapsed_time

# Histogram binnings
columns_binnings = table_dl1_datacheck.histogram_binning.colnames
columns_hist = [col for col in table_dl1_datacheck.cosmics.colnames if "hist_" in col]
for colname in columns_binnings:
    _bins_ = table_dl1_datacheck.histogram_binning.col(colname)[0]
    if len(_bins_.shape) == 1: # We get the binning if is simply an array (1-dimensional)
        globals()[colname.replace("hist", "bins")] = _bins_
        # Computing central bins, in order to do it, we check for logarithmic scales
        if np.sum(np.diff(np.diff(_bins_))) < 1e-14:
            globals()[colname.replace("hist", "bins") + "_center"] = (_bins_[1:] + _bins_[:-1]) / 2
        else:
            globals()[colname.replace("hist", "bins") + "_center"] = (_bins_[1:] * _bins_[:-1]) ** 0.5

for colname in columns_hist:
    counts = table_dl1_datacheck.cosmics.col(colname)
    if len(counts.shape) == 2: # We get the binning if is simply N arrays (1-dimensional)
        globals()[colname] = counts

In [ ]:
# Reading the cuts used from DL3


In [ ]:
hist_width[0]

In [ ]:
for col in columns_hist:
    print(col)
    bins = globals()[col.replace("hist", "bins")+"_center"]
    plt.title(col)
    plt.plot(bins, globals()[col][10])
    if np.sum(np.diff(np.diff(bins))) > 1e-14:
        plt.loglog()
    plt.show()
# plt.xlim(4, 200)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))

ax.plot(srun_number, srun_elapsed_time, ds="steps-mid")

# ax.set_yscale("log")
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))

ax.plot(srun_cumulative_time, srun_rates, ds="steps-post")
ax.plot(srun_cumulative_time, srun_rate_cleaned_events, ds="steps-post")

# ax.set_yscale("log")
ax.grid()
plt.show()

In [ ]:
file_dl1_datacheck.cosmics.col("charge_mean")

In [ ]:
file_dl1_datacheck.cosmics

In [ ]:
file_dl1_datacheck.cosmics.col("dragon_time")

In [ ]:
# Source data
source_coordinates = SkyCoord.from_name(source_name, frame="icrs")
source_ra, source_dec = source_coordinates.ra.deg, source_coordinates.dec.deg

In [ ]:
plt.plot(srun_tel_ra, srun_tel_dec, ".");

In [ ]:
file_dl1_datacheck.dl1datacheck.cosmics.col("num_ucts_jumps")

In [ ]:
pd.read_hdf(path_dl1_dcheck, "/dl1datacheck/cosmics")

In [ ]:
tab = pd.read_hdf(path_dl1_scaled, "/dl1/event/telescope/parameters/LST_LSTCam")

In [ ]:
tab

In [ ]:
find_run_datacheck(run_number)

In [ ]:
path_dl1_datacheck = 